<a href="https://colab.research.google.com/github/hurileborjigin/pytorch_notes/blob/main/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Design Model
## 2) Construct loss and optimizer
## 3) Training loop
  - 1. forward pass
  - 2. backward pass
  - 3. update weights


In [6]:
# Impport packages
import torch
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np


In [43]:
# Prepare datasets

bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

num_samples, num_features = X.shape


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Scale the feature
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))

y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))


y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)



In [44]:
# Set up the model
# f = sigmoid(w * x + b)
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super().__init__()
    self.linear = nn.Linear(n_input_features, 1)

  def forward(self, x):
    y_pred = torch.sigmoid(self.linear(x))

    return y_pred


In [45]:
# Set up the loss
model = LogisticRegression(num_features)
criterion = nn.BCELoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



In [46]:
# Actual trianing loop

epochs = 10000
for i in range(epochs):
  # forward pass
  y_pred = model(X_train)
  loss = criterion(y_pred, y_train)

  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  if(i+1) % 1000 == 0:
    print(f'Epoch: {i + 1}, loss = {loss}' )


Epoch: 1000, loss = 0.02601570077240467
Epoch: 2000, loss = 0.018966784700751305
Epoch: 3000, loss = 0.013820460997521877
Epoch: 4000, loss = 0.009663179516792297
Epoch: 5000, loss = 0.006513688247650862
Epoch: 6000, loss = 0.004247621167451143
Epoch: 7000, loss = 0.0027019537519663572
Epoch: 8000, loss = 0.001688332762569189
Epoch: 9000, loss = 0.001042284769937396
Epoch: 10000, loss = 0.000639330071862787


In [47]:
# Evaluation
with torch.no_grad():
  y_pred = model(X_test)
  y_pred_cls = y_pred.round()
  acc = y_pred_cls.eq(y_test).sum()/ float(y_test.shape[0]) * 100
  print(f'Accuracy: {acc:.4f}%')

Accuracy: 96.4912%
